# Initialization

In [43]:
import os
import logging
os.environ['SOURCE_FILE']='C:/workspace/experiments/GenAI/Projects/Test Data/aws.pdf'
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

# Load data from source

In [44]:
from langchain_community.document_loaders import PyMuPDFLoader
f = os.getenv('SOURCE_FILE')
logger.info(f"loading file: {f}")
loader = PyMuPDFLoader(f)
docs = loader.load()

INFO:__main__:loading file: C:/workspace/experiments/GenAI/Projects/Test Data/aws.pdf


# Split data

In [45]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
transformed_docs = text_splitter.split_documents(docs)

# Extract terms from chunks and save as metadata

In [53]:
from langchain.schema.document import Document
import nltk
from rake_nltk import Rake

nltk.download('stopwords')
nltk.download('punkt')
r = Rake()
transformed_docs_with_terms = []

for doc in transformed_docs:
    r.extract_keywords_from_text(doc.page_content)
    keywords = r.get_ranked_phrases()
    transformed_docs_with_terms.append(Document(page_content=doc.page_content, metadata={**doc.metadata, 'file_name': os.path.basename(doc.metadata['file_path']), 'terms': keywords}))

logger.info(f"total chunks - {len(transformed_docs_with_terms)}")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\samriddha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\samriddha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
INFO:__main__:total chunks - 42


# Initialize embedding function

In [54]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(openai_api_key=os.getenv('OPENAI_API_KEY'), model="text-embedding-ada-002")

# Create OpenSearch Index

In [55]:
mappings = {
    "properties": {
        "id": {
            "type": "text"
        },
        "file_name": {
            "type": "keyword"
        },
        "vector_field": {
            "type": "knn_vector",
            "dimension": 1536,
            "method": {
                "engine": "nmslib",
                "space_type": "l2",
                "name": "hnsw",
                "parameters": {
                  "ef_construction": 512,
                  "m": 16
                }
            }
        },
        "msmarco_distilbert_embedding": {
            "type": "knn_vector",
            "dimension": 768,
            "method": {
                "engine": "lucene",
                "space_type": "l2",
                "name": "hnsw",
                "parameters": {}
            }
        },
        "text": {
            "type": "text"
        },
        "terms": {
            "type": "text"
        }
    }
}


In [62]:
from opensearchpy import OpenSearch
from uuid import uuid4
import time

host = 'localhost'
port = 9200
auth = ('admin', 'Severus11#')

index_name = 'aws-idx'
index_body = {
  "settings": {
    "index.knn": True,
    "default_pipeline": "msmarco-distilbert-base-tas-b-ingest-pipeline"
  },
  "mappings": mappings
}

client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    use_ssl = False,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False
)
response = client.indices.create(index_name, body=index_body)
logger.info(response)

for doc in transformed_docs_with_terms:
    id = str(uuid4())
    d = {
        'id': id,
        'text': doc.page_content,
        **doc.metadata
    }
    response = client.index(
        index = 'aws-idx',
        body = d,
        id = id,
        refresh = True
    )
    time.sleep(15)

INFO:opensearch:PUT http://localhost:9200/aws-idx [status:200 request:0.269s]
INFO:__main__:{'acknowledged': True, 'shards_acknowledged': True, 'index': 'aws-idx'}
INFO:opensearch:PUT http://localhost:9200/aws-idx/_doc/3705d69a-ba27-4aec-a8a3-429fdb228558?refresh=true [status:201 request:0.517s]
INFO:opensearch:PUT http://localhost:9200/aws-idx/_doc/347830b7-f234-4401-a585-4d4976d29167?refresh=true [status:201 request:0.296s]


TransportError: TransportError(500, 'm_l_exception', 'Failed to inference TEXT_EMBEDDING model: BvRH_I8BLuXZPFPYQadN')

# Load data to OpenSearch Vector Store

In [57]:
from langchain_community.vectorstores import OpenSearchVectorSearch
OpenSearchVectorSearch.from_documents(
    transformed_docs_with_terms,
    embeddings,
    index_name="aws-idx",
    opensearch_url="http://localhost:9200",
    http_auth=("admin", "Severus11#"),
    use_ssl = False,
    verify_certs = False,
    timeout=300,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:opensearch:GET http://localhost:9200/aws-idx [status:200 request:0.022s]
INFO:opensearch:POST http://localhost:9200/_bulk [status:200 request:10.214s]


BulkIndexError: ('7 document(s) failed to index.', [{'index': {'_index': 'aws-idx', '_id': 'ffd3af72-9577-4b5e-8d0e-788847b96223', 'status': 500, 'error': {'type': 'm_l_exception', 'reason': 'Failed to inference TEXT_EMBEDDING model: BvRH_I8BLuXZPFPYQadN', 'caused_by': {'type': 'privileged_action_exception', 'reason': None, 'caused_by': {'type': 'translate_exception', 'reason': 'ai.djl.engine.EngineException: The following operation failed in the TorchScript interpreter.\nTraceback of TorchScript, serialized code (most recent call last):\n  File "code/__torch__/sentence_transformers/SentenceTransformer.py", line 14, in forward\n    input_ids = input["input_ids"]\n    mask = input["attention_mask"]\n    _2 = (_0).forward(input_ids, mask, )\n          ~~~~~~~~~~~ <--- HERE\n    _3 = {"input_ids": input_ids, "attention_mask": mask, "token_embeddings": _2, "sentence_embedding": (_1).forward(_2, )}\n    return _3\n  File "code/__torch__/sentence_transformers/models/Transformer.py", line 11,